---
title: "Notebook to explore develop backprop from first principles"
author: "John Richmond"
format:
  html:
    code-fold: False
  pdf:
    geometry:
      - top=30mm
      - left=20mm
jupyter: python3
toc: true
number-sections: False
shift-heading-level-by: -1
---

## Indroduction

Notebook based upon the fastai course 22p, "Part 2 of Practical Deep Learning for Coders".  The intent is to build the capability for back propagation used in frameworks such as pytorch

## The forward and backward passes

Load data and setup paths.  The data is the mnist dataset with the training dataset having 50K 28x28 greyscale images, with the classification being the digit, hence 10 classes.  The validation dataset has 10k images

In [1]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor
from fastcore.test import test_close
torch.manual_seed(42)

mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)

path_to_data = Path('/Users/johnrichmond/datasets') / Path('data')
path_to_data.mkdir(exist_ok=True)
path_gz = path_to_data / 'mnist.pkg.gz'

with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

### Basic architecture

In this case the final value is not the class but an integer intended to have the same value as the class.  This is a crude model and will not work very well but helps to develop methodology

In [2]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

Create a two hidden layers with weight and bias

In [7]:
# num hidden - The depth of the first weights layer
nh = 50

In [8]:
w1 = torch.randn(m,nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)
b2 = torch.zeros(1)

In [9]:
# Create a forward pass through the weight layer
def lin(x, w, b): return x@w + b

In [10]:
# Pass the validation dataset through the first layer
t = lin(x_valid, w1, b1)
t.shape

torch.Size([10000, 50])

In [12]:
# Create a non-linear activation layer (relu)
def relu(x): return x.clamp_min(0.)

In [13]:
# Pass the output from the first layer through the activation
t = relu(t)
t

tensor([[ 0.00,  0.00,  0.00,  ...,  4.04,  0.00,  0.00],
        [ 3.56,  0.00,  0.00,  ...,  0.88,  3.10,  0.00],
        [ 6.66,  4.48,  0.00,  ...,  5.81,  8.84,  3.07],
        ...,
        [16.45,  0.00,  0.00,  ..., 11.17,  0.00,  0.00],
        [ 7.03,  0.51,  0.00,  ...,  8.39,  0.00,  3.69],
        [10.94,  0.00,  0.00,  ...,  0.00,  0.00, 11.96]])

Now create a simple model that passes the data through the first layer and the activation layer and then the second layer.  There is no final non-linear activation

In [14]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    return lin(l2, w2, b2)

In [15]:
res = model(x_valid)
res.shape

torch.Size([10000, 1])

### Loss function

Initially MSE will be used, this will be replaced shortly by a more suitable classification metic, but once again, it allows the methodology to be developed

In [12]:
res.shape,y_valid.shape

(torch.Size([10000, 1]), torch.Size([10000]))

In [17]:
# Generate the difference between the predicted value and the actual
(res[:, 0]-y_valid).shape

torch.Size([10000])

Convert the train and valid datasets to floats and pass through the model

In [18]:
y_train,y_valid = y_train.float(),y_valid.float()

preds = model(x_train)
preds.shape

torch.Size([50000, 1])

In [20]:
# Create mse function
def mse(output, targ): return (output[:,0]-targ).pow(2).mean()

In [22]:
# Create loss value with the training dataset
mse(preds, y_train)

tensor(1549.37)

### Gradients and backward pass
sympy is a useful library that allows symbolic differentiation

In [24]:
#%pip install sympy

In [25]:
from sympy import symbols,diff
x,y = symbols('x y')
diff(x**2, x)

2*x

In [26]:
diff(3*x**2+9, x)

6*x

In [31]:
w1.shape

torch.Size([784, 50])

In [29]:
x_valid.shape

torch.Size([10000, 784])

In [33]:
l1 = x_valid @ w1 +b1
l1.shape, w1.shape

(torch.Size([10000, 50]), torch.Size([784, 50]))

In [34]:
l2 = relu(l1)

In [36]:
out = l2@w2 + b2
out.shape

torch.Size([10000, 1])

In [38]:
diff = out[:,0] - y_valid
diff.shape

torch.Size([10000])

In [39]:
loss = diff.pow(2).mean()

In [40]:
loss, loss.shape

(tensor(1595.00), torch.Size([]))

In [42]:
out.g = 2.*diff[:,None] / x_valid.shape[0]
out.g.shape

torch.Size([10000, 1])

In [55]:
x_in = l2.clone()

In [56]:
x_in.g = out.g @ w2.t()
x_in.g.shape

torch.Size([10000, 50])

In [57]:
test = x_in.unsqueeze(-1)
test.shape

torch.Size([10000, 50, 1])

In [58]:
test2 = out.g.unsqueeze(1)
test2.shape

torch.Size([10000, 1, 1])

In [59]:
tmp = (x_in.unsqueeze(-1) * out.g.unsqueeze(1))
tmp.shape

torch.Size([10000, 50, 1])

In [60]:
w2.g = tmp.sum(0)
w2.g.shape

torch.Size([50, 1])

Create a function to calculate the gradient of the linear layer.  The gradient of y=ax+b is simply a, hence the gradient dy/dx=a, The output loss is mse, hence the differencial l = (pred - targ).sum/num_samples

Add a section to cover why the input gradient is the output gradient * the transpose of the weight matrix

Further work is needed to establish the basis of the gradient calculation

In [64]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    #import pdb; pdb.set_trace()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [65]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    diff = out[:,0]-targ
    loss = diff.pow(2).mean()
    
    # backward pass:
    out.g = 2.*diff[:,None] / inp.shape[0]
    lin_grad(l2, out, w2, b2)
    l1.g = (l1>0).float() * l2.g
    lin_grad(inp, l1, w1, b1)

In [66]:
forward_and_backward(x_train, y_train)

In [67]:
# Save for testing against later
def get_grad(x): return x.g.clone()
chks = w1,w2,b1,b2,x_train
grads = w1g,w2g,b1g,b2g,ig = map(get_grad, chks)

We cheat a little bit and use PyTorch autograd to check our results.

In [68]:
def mkgrad(x): return x.clone().requires_grad_(True)
ptgrads = w12,w22,b12,b22,xt2 = map(mkgrad, chks)

In [69]:
def forward(inp, targ):
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    return mse(out, targ)

In [70]:
loss = forward(xt2, y_train)
loss.backward()

In [71]:
for a,b in zip(grads, ptgrads): test_close(a.grad, b, eps=0.01)

## Refactor model

### Layers as classes

In [72]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [73]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out

    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [74]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

Note that the below works because the loss gradient calculated by the loss.backward (self.inp.g) is assigned to self.inp, which is the same array as is passed in from the last layer., in this case the self.out of the last linear layer.  This seems a flaky way to achieve it

In [84]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        #import pdb; pdb.set_trace()
        return self.loss(x, targ)
    
    def backward(self):
        import pdb; pdb.set_trace()
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [85]:
model = Model(w1, b1, w2, b2)

In [86]:
loss = model(x_train, y_train)

In [79]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

### Module.forward()

Note that compared to the above the input and output are passed into the backward method and not held totally within.  There are a number of important things that are done below:

* The args are all saved when the module is called, hence they are avilable when the backward method is called.  These are stores as a list and in order and can be recalled from self.args
* The output from the layer is also stored pas well as being passed onto the calling function

In [174]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out

    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)
    def bwd(self): raise Exception('not implemented')

In [175]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [176]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
    def forward(self, inp): return inp@self.w + self.b
    def bwd(self, out, inp):
        inp.g = self.out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [177]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): 
        #import pdb; pdb.set_trace()
        inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [178]:
class Model(Module):
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
        
    def forward(self, x, targ):
        for layer in self.layers: x = layer(x)
        self.loss(x, targ)   
        return self.loss
    
    def backward(self):
        self.loss.backward()
        # x should now have a .g property
        #import pdb; pdb.set_trace()
        for l in reversed(self.layers):
            l.backward()

In [179]:
model = Model(w1, b1, w2, b2)

In [180]:
loss = model(x_train, y_train)

In [181]:
model.backward()

In [182]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

### Autograd

In [183]:
from torch import nn
import torch.nn.functional as F

In [184]:
class Linear(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.w = torch.randn(n_in,n_out).requires_grad_()
        self.b = torch.zeros(n_out).requires_grad_()
    def forward(self, inp): return inp@self.w + self.b

In [185]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [Linear(n_in,nh), nn.ReLU(), Linear(nh,n_out)]
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return F.mse_loss(x, targ[:,None])

In [186]:
model = Model(m, nh, 1)
loss = model(x_train, y_train)
loss.backward()

In [187]:
l0 = model.layers[0]
l0.b.grad

tensor([ -3.30,  -0.11,   5.82,  -3.41,   0.39,   0.35,  -1.73,  -3.42,  -5.44,  14.03,  20.84,   3.44,   0.98,  -9.51,
        -25.30,  -0.86,  12.95,  -3.85,  20.71,   3.13,  -1.34,   3.00,   1.82,   1.46,  -8.74,  -2.52,  -1.46,  -3.54,
          2.74,  -3.57,  19.23,   9.41,   0.59,  -4.11,  -2.01,  -6.21,   9.13,   6.59,   1.45,   0.05,  -0.87,  -0.28,
          0.49,   1.72,   2.44,  -1.45,   0.81,   5.45,   3.13,   0.20])